In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import numpy as np



C:\Users\Chandranil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import csv
import sys
import pandas as pd
import os
import glob

path  = r"D:\Traces\Alldata" 
path2  = r"D:\Traces\Alldata\output"
filename_wr = os.path.join(path2,'output.csv')

#filename_rd = os.path.join(path,'A.trace.csv')
names = ['Operation', 'TimeStamp', 'ProcessName', 'ThreadID', 'IrpPtr' , 'ByteOffset', 'IOSize','ElapsedTime', 'DiskNum', 'IrpFlags', 'DiskSvcTime', 'IOPri' , ' VolSnap', 'FileObject', ' FileName']

#df = pd.read_csv(filename_rd,engine='python',header=None,skiprows = 73,names=names,na_values=['-1'], index_col=False)


all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
print(all_files)
df_from_each_file = (pd.read_csv(f,engine='python',header=None,names=names,na_values=['-1'], index_col=False) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)


concatenated_df.columns = names



headers = list(concatenated_df.columns.values)
print(headers)

#skiprows=lambda x: x % 2 != 0

['D:\\Traces\\Alldata\\output_1.csv', 'D:\\Traces\\Alldata\\output_2.csv', 'D:\\Traces\\Alldata\\output_3.csv', 'D:\\Traces\\Alldata\\output_4.csv']
['Operation', 'TimeStamp', 'ProcessName', 'ThreadID', 'IrpPtr', 'ByteOffset', 'IOSize', 'ElapsedTime', 'DiskNum', 'IrpFlags', 'DiskSvcTime', 'IOPri', ' VolSnap', 'FileObject', ' FileName']


In [3]:
print("Before drop: {}".format(concatenated_df.columns))

concatenated_df.drop(concatenated_df.columns[[8,9,10,11,12,13,14]], axis=1, inplace=True)
print("After drop: {}".format(concatenated_df.columns))

Before drop: Index(['Operation', 'TimeStamp', 'ProcessName', 'ThreadID', 'IrpPtr',
       'ByteOffset', 'IOSize', 'ElapsedTime', 'DiskNum', 'IrpFlags',
       'DiskSvcTime', 'IOPri', ' VolSnap', 'FileObject', ' FileName'],
      dtype='object')
After drop: Index(['Operation', 'TimeStamp', 'ProcessName', 'ThreadID', 'IrpPtr',
       'ByteOffset', 'IOSize', 'ElapsedTime'],
      dtype='object')


In [4]:
concatenated_df.shape

(1031881, 8)

In [5]:
print("Starting file:")
print(concatenated_df[0:10])

print("Ending file:")
print(concatenated_df[-10:])

Starting file:
   Operation  TimeStamp  ProcessName  ThreadID               IrpPtr  \
0  Operation  TimeStamp  ProcessName  ThreadID               IrpPtr   
1          0     153302      p1 ( 4)        36   1.2740074430687651   
2          0     153756   p0 (21316)      7552   0.8872597941837778   
3          0     160029   p0 (21316)      7552   0.8872597941837778   
4          0     160057      p1 ( 4)        36   1.0351586121885508   
5          0     165085   p0 (21316)      7552  -1.1799920460750726   
6          0     171596   p0 (21316)      7552  -1.1799920460750726   
7          0     177048   p0 (21316)      7552   1.2740074430687651   
8          0     182217   p0 (21316)      7552  -1.1849066994192514   
9          0     184062   p0 (21316)      7552  -1.1849066994192514   

             ByteOffset                IOSize           ElapsedTime  
0            ByteOffset                IOSize           ElapsedTime  
1  -0.17570138803562596  -0.17570138803562596    0.199634804833

In [6]:
df = concatenated_df

In [7]:
df.shape

(1031881, 8)

In [8]:
count =-1

for index, row in df.iterrows():
    a2 = row['Operation']
    count=count+1

    if (a2.strip() == 'DiskRead'):
        df.at[index, 'Operation'] = 0
    elif (a2.strip() == 'DiskWrite'): 
        df.at[index, 'Operation'] = 1
    else :
        df.drop(count,inplace= True)

        

KeyboardInterrupt: 

In [ ]:
print("Ending file:")
print(concatenated_df[-10:])

In [ ]:
df.to_csv(filename_wr,index=False)

In [ ]:
# Data Filtering for IrpPte

 
for index, row in df.iterrows():
    a = (row['IrpPtr'])
    a1 = int(a,16) - 18440000000000000000
    df.at[index, 'IrpPtr'] = a1
    b = (row['IOSize'])
    b1 = int(b,16) 
    df.at[index, 'IOSize'] = b1
    c = (row['ByteOffset'])
    c1 = int(b,16) 
    df.at[index, 'ByteOffset'] = c1

filename_wr = os.path.join(path2,'output.csv')
print(df[0:10])


In [ ]:
df.to_csv(filename_wr,index=False)
df = pd.read_csv(filename_wr,engine='python',index_col=False)
df.dtypes

In [ ]:
df['IOSize'] = df['IOSize']/512
df['ByteOffset'] = df['ByteOffset']/512

In [ ]:
#Normalizing IO Size
mean = df['IOSize'].mean(axis=0)
df['IOSize'] -= mean
std = df['IOSize'].std(axis=0)
df['IOSize'] /= std

In [ ]:
#Normalizing IrpPtr
mean = df['IrpPtr'].mean(axis=0)
df['IrpPtr'] -= mean
std = df['IrpPtr'].std(axis=0)
df['IrpPtr'] /= std


In [ ]:
print(df[10:50])

In [ ]:
#Normalizing ByteOffset
mean = df['ByteOffset'].mean(axis=0)
df['ByteOffset'] -= mean
std = df['ByteOffset'].std(axis=0)
df['ByteOffset'] /= std


In [ ]:
df.dtypes

In [ ]:
df.to_csv(filename_wr,index=False)

In [ ]:
df = pd.read_csv(filename_wr,engine='python',index_col=False)
df.dtypes

In [ ]:
#Normalizing Elapsed Time
mean = df['ElapsedTime'].mean(axis=0)
df['ElapsedTime'] -= mean
std = df['ElapsedTime'].std(axis=0)
df['ElapsedTime'] /= std

In [ ]:
df.sort_values(by=['TimeStamp'])

In [ ]:
df.to_csv(filename_wr,index=False)

In [ ]:
print("Done!")

In [ ]:
df.shape

In [ ]:
print(filename_wr)